In [1]:
#!/usr/bin/python
# -*- coding:utf8 -*-

import os
import time
import re
import jieba.analyse

In [2]:
def post_cut(url):
    fr = open(url + "/post_data.txt",encoding='utf8')
    fo = open(url + "/post_key.txt", "a+",encoding='utf8')
    jieba.analyse.set_stop_words(url + "/stopwords.txt")
    for line in fr.readlines():
        term = line.strip().split("\t")
        if len(term) == 3 and term[2] != "":
            key_list = jieba.analyse.extract_tags(term[2], 30)  # get keywords
            ustr = term[0] + "\t"
            for i in key_list:
                # ustr += i.encode("utf-8") + " "
                ustr += i + " "
            fo.write(ustr + "\n")
    fr.close()
    fo.close()

In [3]:
def post_tfidf(url):
    from sklearn.feature_extraction.text import HashingVectorizer
    fr = open(url + "/post_key.txt",encoding='utf8')
    id_list = []
    data_list = []
    for line in fr.readlines():
        term = line.strip().split("\t")
        if len(term) == 2:
            id_list.append(term[0])
            data_list.append(term[1])

    hv = HashingVectorizer(n_features=10000, non_negative=True)  # 该类实现hash技巧
    post_tfidf = hv.fit_transform(data_list)  # return feature vector 'fea_train' [n_samples,n_features]
    print('Size of fea_train:' + repr(post_tfidf.shape))
    print(post_tfidf.nnz)
    post_cluster(url, id_list, post_tfidf)

In [4]:
def post_cluster(url, id, tfidf_vec):
    from sklearn.cluster import KMeans
    kmean = KMeans(n_clusters=300)
    print("kmeans")
    kmean.fit(tfidf_vec)
    print("kmeans1")
    #     pred = kmean.transform(tfidf_vec)

    #   count1 = 0
    #   count2 = 0
    #     pred_str = []
    #
    #     for item in pred:
    #         count1 += 1
    #         vec = ""
    #         for tmp in item :
    #             vec += str(tmp)[0:7] + "\t"
    #         pred_str.append(vec)
    #
    #     print len(pred_str)
    #     print len(id)
    count2=0
    pred = kmean.predict(tfidf_vec)
    fo = open(url + "/cluster.txt", "a+",encoding='utf8')
    for i in range(len(pred)):
        count2 += 1
        fo.write(id[i] + "\t" + str(pred[i]) + "\n")
    fo.close()
    print("kmeans2")
    print("%d" %count2)

In [5]:
def post_lda(url, cluster):
    from gensim import corpora, models, matutils
    count = 0
    fr = open(url + "/post_key.txt",encoding='utf8')
    fo2 = open(url + "/post_vec_lda.txt", "a+",encoding='utf8')
    id_list = []
    data_list = []

    for line in fr.readlines():
        term = line.strip().split("\t")
        if len(term) == 2:
            count += 1
            id_list.append(term[0])
            word = term[1].strip().split()
            data_list.append(word)
    print("lda")
    dic = corpora.Dictionary(data_list)  # 构造词典
    corpus = [dic.doc2bow(text) for text in data_list]  # 每个text 对应的稀疏向量
#     print("corpus is:")
#     print(corpus)
    tfidf = models.TfidfModel(corpus)  # 统计tfidf
#     print("lda")
    corpus_tfidf = tfidf[corpus]  # 得到每个文本的tfidf向量，稀疏矩阵
#     print("corpus_tfidf is:")
#     print(corpus_tfidf)
    lda = models.LdaMulticore(corpus_tfidf, id2word=dic, workers = 3,num_topics=200)
    lda_1 = 
    corpus_lda = lda[corpus_tfidf]  # 每个文本对应的LDA向量，稀疏的，元素值是隶属与对应序数类的权重
    print("lda is:")
    print(lda)
#     print("lda")

    num = 0
    for doc in corpus_lda:
        wstr = ""
        for i in range(len(doc)):
            item = doc[i]
            wstr += str(item[0]) + "," + str(item[1])[0:7] + "/"
        fo2.write(id_list[num] + "\t" + wstr[0:-1] + "\n")
        num += 1
    fr.close()
    fo2.close()
    print(num)

    if cluster:
        lda_csc_matrix = matutils.corpus2csc(corpus_lda).transpose()  # gensim sparse matrix to scipy sparse matrix
        post_cluster(url, id_list, lda_csc_matrix)

In [6]:
if __name__ == "__main__":
    url = "E:\pythonNotebook\data"
    time1 = time.time()
    post_cut(url)
    post_tfidf(url)
    lda_cluster = False
    post_lda(url, lda_cluster)

    print(time.time() - time1)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\73722\AppData\Local\Temp\jieba.cache
Loading model cost 0.818 seconds.
Prefix dict has been built succesfully.


Size of fea_train:(73522, 10000)
701341
kmeans
kmeans1
kmeans2
73522


D:\Users\73722\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


lda
lda is:
LdaModel(num_terms=92342, num_topics=200, decay=0.5, chunksize=2000)
73522
1769.8138892650604


In [7]:
time1

1520328059.3805363

In [8]:
fr

NameError: name 'fr' is not defined

In [2]:
import gc
import pandas as pd

In [3]:
import numpy as np

In [49]:
url = url = "E:\pythonNotebook\data"
pd.set_option('max_colwidth', 10000)
post_data = pd.read_table(url+ "/post_data.txt",encoding='utf8',index_col=False)

In [50]:
post_data

,id,title,content
0,3,NaN,给老婆过了一个简单的生日，happy birthday 。
1,15,NaN,敢从上面滑下来么？
2,23,NaN,第一次冰钓
3,39,NaN,如果有一天失业了～～那我还能混口饭吃
4,109,NaN,—— 儿 童 节 —— 国际儿童节定于每年的6月1日。为了悼念1942年6月10日的利迪策惨案 和全世界所有在战争中死难的儿童，反对虐杀和毒害儿童，以及保障儿童权利，它是为了保障世界各国儿童的 生存权 、保健权和 受教育权 ， 抚养权 ，为了改善儿童的生活，为了反对虐杀儿童和毒害儿童而设立的 节日 。目前世界上许多 国家 都将6月1日定为 儿童 的 节日 。 从1949年开始，中华人民共和国 正式定每年6月1日为国际儿童节。学校一般会为此组织相关的集体活动，并要求学生正式着装。 香港 虽属共和国一部份，但民间在约定俗成下，香港儿童节的日期与中国 台湾 的儿童节日期一样，即4月4日。民间庆祝的方式多以送玩具 礼物 给小朋友，或陪小孩出外吃大餐或游玩。 —— 庆 祝 —— 韩国的儿童节开始于1923年，是从“男孩节”演变过来的。这也是韩国的公众假日，定在每年的5月5日。家长们通常会在这一天带孩子去公园、动物园或者其他游乐设施，让孩子开心地度过假日。 日本的儿童节，叫做“儿童日”，是一个传统节日。在每年的5月5日，日本的家庭都会庆祝孩子的长大。二战前，这个节日被称为“端午节”，并且只是男孩的节日。在节日当天，日本的家庭都会在屋顶上悬挂鱼状的标志，用来象征儿童消除厄运，克服困难，顺利成长。 不过日本在3月3日还有单独的女孩节，从上巳节发展过来，女儿节以摆放各种玩偶来庆祝。男儿节悬挂鲤鱼旗是源自中国的“望子成龙”和“鲤鱼跳龙门”的传说。 大多数 伊斯兰 国家都将斋月后第14天定为“糖果节”，对孩子们来说，这也是最快乐的儿童节。“糖果节”一般为期三天，小朋友们三五成群，到附近的各家各户索要糖果。按照当地 民俗 ，大人们不能拒绝儿童的要求，所以孩子们这一天总是能满载而归，欢天喜地。 非洲西部的儿童节是疯狂的，他们的国家大都有专门的“儿童狂欢节”，而且往往持续一个月。非洲人历来能歌善舞，在“儿童狂欢节”里，尽管人们的生活条件各不相同，但所有孩子都会尽情欢乐，热闹非凡。 不过，比起上述国家的儿童，伊拉克的孩子们就可怜多了，当许多国家的小朋友在欢庆儿童节的时候，伊拉克儿童大多根本不知道世界上还有一个属于他们自己的节日。他们仍一如既往，或在学校上课，或在作坊打工，或沿街乞讨。 一些伊拉克人透露，在萨达姆执政的20多年里，6月1日被定为伊拉克的石油国有化日。每到这一天，伊拉克全国要举办各种庆祝活动，但与儿童无关，所以孩子们就没有过过儿童节。 在被忽略的儿童节里，伊拉克幼儿园、小学里没有任何节日气氛，自然也不会有庆祝活动。 图片来源网络
5,155,NaN,"五月榴花妖艳烘，绿杨带雨垂垂重，五色新丝缠角粽。金盘送，生绡画扇盘双凤。 正是浴兰时节动，菖蒲酒美清尊共，叶里黄鹂时一弄。犹瞢忪，等闲惊破纱窗梦。 ——（宋）欧阳修《渔家傲·五月榴花妖艳烘》 端 午 节 端午节（Dragon Boat Festival）为每年农历五月初五，又称端阳节、午日节、五月节等。“端午节”为中国国家法定节假日之一，并已被列入世界非物质文化遗产名录。 端午节起源于中国，最初是中国人民祛病防疫的节日，吴越之地春秋之前有在农历五月初五以龙舟竞渡形式举行部落图腾祭祀的习俗；后因诗人屈原在这一天死去，便成了中国汉族人民纪念屈原的传统节日；部分地区也有纪念伍子胥、曹娥等说法。 习 俗 过端午节，是中国人二千多年来的传统习惯，由于地域广大，民族众多，加上许多故事传说，于是不仅产生了众多相异的节名，而且各地也有着不尽相同的习俗。其内容主要有：女儿回娘家，挂钟馗像，迎鬼船、躲午，帖午叶符，悬挂菖蒲、艾草，游百病，佩香囊，备牲醴,赛龙舟，比武，击球，荡秋千，给小孩涂雄黄，饮用雄黄酒、菖蒲酒，吃五毒饼、咸蛋、粽子和时令鲜果等。 挂草药 挂艾草、菖蒲、榕枝 艾、菖蒲和蒜被称为“端午三友”。南北朝时，端午又称为“沐兰节”，荆楚一带有采艾的习俗。采艾要在鸡未鸣以前就出发，挑选最具人形的艾草带回去挂在门上，有的还将艾草扎成虎形，再粘贴艾叶于其上。艾与菖蒲中含有芳香油，它们和蒜一样都有杀菌作用。古时，人们缺乏科学观念，误以为疾病皆由鬼邪作祟所至，于是，端午节这天，人们以菖蒲作宝剑，以艾作鞭子，以蒜头作锤子，又称为“三种武器”，认为可以退蛇、虫、病菌，斩除妖魔。 划龙舟 龙舟一词，最早见于先秦古书《穆天子传》卷五：“天子乘鸟舟、龙舟浮于大沼。”预《九歌·湘君》中“驾飞龙今北征，邅吾道兮洞庭”，“石濑浅浅，飞龙兮翩翩”，学者们也认为“飞龙”即龙舟。《湘君》即描写湘人驭驾龙舟，将玉佩沉入江中（与抛粽子入江相仿）悼念某位历史人物之诗。这即与“魂舟”暗合，与楚国《人物御龙帛画》之像暗合，可互为印证。《荆楚岁时记》载：“五月五日，谓之浴兰节。……是日，竞渡，竞采杂药。”。 在划龙船时，又多有唱歌助兴的龙船歌流传。如湖北秭归划龙船时，有完整的唱腔，词曲根据当地民歌与号子融汇而成，唱歌声雄浑壮美，扣人心弦，即“举揖而相和之”之遗风。又如广东南雄县的龙船歌，是在四月龙船下水后唱到端午时止，表现内容十分广泛。 饮 食 粽子 粽子古称“角黍”，传说是为祭投江的屈原而发明的，真正有文字记载的粽子见于晋周处的《风土记》；而流传有序，历史最悠久的粽子则是西安的蜂蜜凉粽子，载于唐韦巨源《食谱》。 “粽子香，香厨房。艾叶香，香满堂。桃枝插在大门上，出门一望麦儿黄。这儿端阳，那儿端阳，处处都端阳。”这是旧时流行甚广的一首描写过端午节的民谣。总体上说，各地人民过端午节的习俗大同小异，而端午节吃粽子，古往今来，中国各地都一样。 雄黄酒 端午饮雄黄酒的习俗，从前在长江流域地区极为盛行。古语曾说：“饮了雄黄酒，病魔都远走。”雄黄是一种矿物质，俗称“鸡冠石”，其主要成分是硫化砷，并含有汞，有毒。一般饮用的雄黄酒，只是在白酒或自酿的黄酒里加入微量雄黄而成，无纯饮的。雄黄酒有杀菌驱虫解五毒的功效，中医还用来治皮肤病。在没有碘酒之类消毒剂的古代，用雄黄泡酒，可以祛毒解痒。未到喝酒年龄的小孩子，大人则给他们的额头、耳鼻、手足心等处涂抹上雄黄酒，意在消毒防病，虫豸不叮。 故 事 三闾大夫屈原的传说 屈原是战国时期楚国贵族出身，任三闾大夫、左徒,兼管内政外交大事。他主张对内举贤能，修明法度，对外力主联齐抗秦。后因遭贵族排挤，被流放沅、湘流域。 公元前278年秦将白起一举攻破楚国首都郢都，并在夷陵（今宜昌）焚烧楚先王陵墓。忧国忧民的屈原在汨罗江怀石自杀，端午节据说就是他的忌日。人们划龙舟去打捞他的尸体，怕鱼虾咬食屈原就向江中投放糯米，后来人们为了纪念他就在这一天划龙舟。 图文来源网络"
6,159,NaN,‍ ‍ ‍ ‍ —— 父 亲 节 —— 父亲节， 顾名思义是感恩父亲的节日。 约始于二十世纪初， 起源于美国， 现已广泛流传于世界各地， 节日日期因地域而存在差异。 最广泛的日期在每年6月的第三个星期， 世界上有52个国家和地区是在这一天过父亲节。 节日里有各种的庆祝方式， 大部分都与赠送礼物、 家族聚餐或活动有关。 中国大陆官方没有设立正式的父亲节。 但内地民众习惯上使用6月第三个星期日当做父亲； 中国台湾父亲节是8月8日。 中国台湾的父亲节订于每年的八月八日， 又称为“ 八八节”。 这是因为“ 八八” 和爸爸相近， 而且“ 八八” 两字连缀起来， 又好象一个 “ 父”， 所以父亲节特别被定于八月八日。 —— 起 源 —— 父亲节是由约翰·布鲁斯·多德夫人倡议成立的。多德夫人早年丧母，她有5个弟弟，姐弟6人的生活负担全落到了父亲身上。父亲每天起早贪黑，无微不至地关心着孩子们的成长，自己则过着节衣缩食的节俭日子。多德长大以后，根据自己的切身体会，深感父亲这种自我牺牲的精神应该得到表彰。 做父亲的也应该像母亲们那样，有一个让全社会向他们表示敬意的节日。于是，她给华盛顿州政府写了一封言辞恳切的信，建议以她父亲的生日，每年的6月5日作为父亲节。州政府采纳了这一建议，但把节期改在每年6月的第三个星期日。1972年，在各方的强烈呼吁下，美国总统尼克松签署了建立父亲节的议会决议，使其成为全国性的节日。 —— 风 俗 习 惯 —— 在父亲节这天， 人们选择特定的鲜花来表示对父亲的敬意。 人们采纳了杜德夫人建议， 佩戴红玫瑰向健在的父亲们表示爱戴， 佩戴白玫瑰对故去的父亲表示悼念。 后来在温哥华， 人们选择了佩戴白丁香， 宾夕法尼亚人用蒲公英向父亲表示致意。 红色或白色玫瑰是公认的父亲节的节花。 父亲节在全美国作为节日确定下来， 比母亲节经过的时间要长一些。 因为建立父亲节的想法很得人心， 所以商人和制造商开始看到商机。 他们不仅鼓励做儿女的给父亲寄贺卡， 而且鼓动他们买领带、 袜子之类的小礼品送给父亲， 以表达对父亲的敬重。 另外， 在父亲节的那一顿早餐是由子女们做的， 父母早上不必早， 可以继续睡， 子女们做好早餐后拿到床前给父母亲用。 图片来源网络 ‍ ‍ ‍ ‍
7,163,NaN,—— 建 军 节 —— 每年的八月一日是中国 人民解放军建军纪念日，因此也叫“ 八一 ”建军节。1933年7月11日， 中华苏维埃共和国临时中央政府 根据 中央革命军事委员会 6月30日的建议，决定8月1日为 中国工农红军 成立纪念日。1949年6月15日，中国人民革命军事委员会发布命令，以“八一”两字作为中国人民解放军军旗和军徽的主要标志。中华人民共和国成立

In [51]:
# cols = post_data.col
s = str(post_data[post_data.id=='109']['content '])
s

'4     —— 儿 童 节 —— 国际儿童节定于每年的6月1日。为了悼念1942年6月10日的利迪策惨案 和全世界所有在战争中死难的儿童，反对虐杀和毒害儿童，以及保障儿童权利，它是为了保障世界各国儿童的 生存权 、保健权和 受教育权 ， 抚养权 ，为了改善儿童的生活，为了反对虐杀儿童和毒害儿童而设立的 节日 。目前世界上许多 国家 都将6月1日定为 儿童 的 节日 。 从1949年开始，中华人民共和国 正式定每年6月1日为国际儿童节。学校一般会为此组织相关的集体活动，并要求学生正式着装。 香港 虽属共和国一部份，但民间在约定俗成下，香港儿童节的日期与中国 台湾 的儿童节日期一样，即4月4日。民间庆祝的方式多以送玩具 礼物 给小朋友，或陪小孩出外吃大餐或游玩。 —— 庆 祝 —— 韩国的儿童节开始于1923年，是从“男孩节”演变过来的。这也是韩国的公众假日，定在每年的5月5日。家长们通常会在这一天带孩子去公园、动物园或者其他游乐设施，让孩子开心地度过假日。 日本的儿童节，叫做“儿童日”，是一个传统节日。在每年的5月5日，日本的家庭都会庆祝孩子的长大。二战前，这个节日被称为“端午节”，并且只是男孩的节日。在节日当天，日本的家庭都会在屋顶上悬挂鱼状的标志，用来象征儿童消除厄运，克服困难，顺利成长。 不过日本在3月3日还有单独的女孩节，从上巳节发展过来，女儿节以摆放各种玩偶来庆祝。男儿节悬挂鲤鱼旗是源自中国的“望子成龙”和“鲤鱼跳龙门”的传说。 大多数 伊斯兰 国家都将斋月后第14天定为“糖果节”，对孩子们来说，这也是最快乐的儿童节。“糖果节”一般为期三天，小朋友们三五成群，到附近的各家各户索要糖果。按照当地 民俗 ，大人们不能拒绝儿童的要求，所以孩子们这一天总是能满载而归，欢天喜地。 非洲西部的儿童节是疯狂的，他们的国家大都有专门的“儿童狂欢节”，而且往往持续一个月。非洲人历来能歌善舞，在“儿童狂欢节”里，尽管人们的生活条件各不相同，但所有孩子都会尽情欢乐，热闹非凡。 不过，比起上述国家的儿童，伊拉克的孩子们就可怜多了，当许多国家的小朋友在欢庆儿童节的时候，伊拉克儿童大多根本不知道世界上还有一个属于他们自己的节日。他们仍一如既往，或在学校上课，或在作坊打工，或沿街乞讨。 一些伊拉克人透露，在萨达姆执政的20多年里，6月1日被定为伊拉克的石油国有化日。每到这一天，伊拉克

In [52]:
len(s)

1111

In [ ]:
models.L